DATASET LINK:-https://components.one/datasets/all-the-news-2-news-articles-dataset/

use this command in the folder in download to delete the line causing error in git bash.

sed '14193d' all-the-news-2-1.csv > all-the-news-2-1-cleaned.csv

open git bash

sed -i '15631d' all-the-news-2-1-cleaned.csv

sed -i '/^$/d' old.csv

awk 'gsub(/"/,"&") % 2 == 0' your_file.csv > cleaned_file.csv........THIS COMMANDS DELETES ROWS NOT HAVING EVEN COUNT OF "

Methoed 1:-Direct Methoed for Bag of Words

#Methoed 1:-Direct Methoed

In [2]:
!pip install dask-ml

In [3]:
import dask.dataframe as dd
from sklearn.feature_extraction.text import CountVectorizer
from dask_ml.feature_extraction.text import HashingVectorizer  # for huge data
from dask_ml.wrappers import ParallelPostFit
import nltk
import pandas as pd
nltk.download('punkt')  # Only if you later tokenize (optional)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:


df = dd.read_csv(
    "all-the-news-2-1-cleaned.csv",
    usecols=['article'],  # or any subset you actually need
    dtype={'article': 'object'},
    on_bad_lines='skip',
    encoding='utf-8'
)

In [5]:
def normalize_text(x):
    return x.lower() if isinstance(x, str) else ''

df["article"] = df["article"].map_partitions(
    lambda col: col.map(normalize_text),
    meta=pd.Series(dtype=object)
)

# 3. Drop missing/empty rows
df = df[df["article"].str.strip() != ""]

# 4. Convert to in-memory list of strings (for scikit-learn)
texts = df["article"].compute()

# 5. Vectorize using CountVectorizer
vectorizer = CountVectorizer(max_features=10000, stop_words='english')  # Limit vocab for memory

X = vectorizer.fit_transform(texts)

print("Shape of vector matrix:", X.shape)  # e.g., (N documents, 10,000 features)
print("Sample feature names:", vectorizer.get_feature_names_out()[:10])


Shape of vector matrix: (297165, 10000)
Sample feature names: ['00' '000' '01' '02' '03' '04' '05' '06' '07' '08']


In [6]:
X_dense_head = X[:5].toarray()  # Only first 5 articles
feature_names = vectorizer.get_feature_names_out()
df_vector_head = pd.DataFrame(X_dense_head, columns=feature_names)

df_vector_head.head()


,00,000,01,02,03,04,05,06,07,08,...,zhang,zika,zimbabwe,zombie,zone,zones,zoo,zte,zuckerberg,zurich
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_tfidf = vectorizer.fit_transform(texts)


In [8]:
X_tfidf_head = X_tfidf[:5].toarray()
feature_names = vectorizer.get_feature_names_out()
df_tfidf_head = pd.DataFrame(X_tfidf_head, columns=feature_names)

df_tfidf_head.head()


,00,000,01,02,03,04,05,06,07,08,...,zhang,zika,zimbabwe,zombie,zone,zones,zoo,zte,zuckerberg,zurich
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
nonzero_docs = (X_tfidf.getnnz(axis=1) > 0).sum()
print(f"Documents with at least 1 TF-IDF word: {nonzero_docs}")
print(vectorizer.get_feature_names_out()[:20])


Documents with at least 1 TF-IDF word: 296894
['00' '000' '01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '100' '101'
 '10th' '11' '110' '11th' '12' '120']


In [10]:
import re
def custom_tokenizer(text):
    tokens = re.findall(r'\b[a-zA-Z]{2,}\b', text.lower())  # only words with 2+ letters
    return tokens

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=10000,
    tokenizer=custom_tokenizer,
    stop_words='english'
)

X_tfidf = vectorizer.fit_transform(texts)


C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [11]:
X_tfidf_head = X_tfidf[:5].toarray()
feature_names = vectorizer.get_feature_names_out()
df_tfidf_head = pd.DataFrame(X_tfidf_head, columns=feature_names)

df_tfidf_head.head()

,aaron,ab,abandon,abandoned,abc,abe,abilities,ability,able,aboard,...,zika,zimbabwe,zombie,zone,zones,zoo,zte,zuckerberg,zuma,zurich
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
